<a href="https://colab.research.google.com/github/kenanmorani/Audio_Signal_Processing_For_Music_Applications_idu/blob/main/Pitch%20Extraction/HW3_audio_signal_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW3 Audio Signal Processing for Music Applications course
Izmir Democracy University, Electrical and Electronics Program    
Submitted to Professor Barsi Bozkurt      
Submitted by Kenan MORANI     

# Installing and importing the required libraries

In [1]:
!pip install essentia mir_eval pysoundfile

     |████████████████████████████████| 12.0MB 335kB/s 
     |████████████████████████████████| 92kB 3.3MB/s 
  Created wheel for mir-eval: filename=mir_eval-0.6-cp36-none-any.whl size=96515 sha256=d461c39c7637b44d529f9e2d5c211e033a2fd2f54fb1a764f9abfea3b3b2f94e
  Stored in directory: /root/.cache/pip/wheels/49/ce/30/730fa72addf275e49d90683b01b3613048b4be3bf7ff8eb6ec
Successfully built mir-eval


In [2]:
import os
from essentia import *
from essentia.standard import *
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import get_window
import soundfile as sf
import urllib.request
from IPython.display import Audio
import mir_eval

In [3]:
#Downloading a standard mir dataset
!pip install mirdata
import mirdata
orchset = mirdata.initialize('orchset')
orchset.download()

     |████████████████████████████████| 6.1MB 5.0MB/s 
     |████████████████████████████████| 184kB 41.8MB/s 
     |████████████████████████████████| 5.6MB 37.7MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
  Created wheel for librosa: filename=librosa-0.8.0-cp36-none-any.whl size=201375 sha256=af05e7d72eab5af6aee2c69d879bd02b396fe7e2c52127c7916d9f02a32b2ddc
  Stored in directory: /root/.cache/pip/wheels/ee/10/1e/382bb4369e189938d5c02e06d10c651817da8d485bfd1647c9
  Created wheel for pretty-midi: filename=pretty_midi-0.2.9-cp36-none-any.whl size=5591954 sha256=d1bf431e9b4c54929b5e87e6435ec133d04679ab82aaf64d32b2ca32b5309011
  Stored in directory: /root/.cache/pip/wheels/4c/a1/c6/b5697841db1112c6e5866d75a6b6bf1bef73b874782556ba66
  Created wheel for jams: filename=jams-0.3.4-cp36-none-any.whl size=64925 sha256=1c53

INFO: Downloading ['all'] to /root/mir_datasets/orchset
INFO: [all] downloading Orchset_dataset_0.zip
311MB [00:20, 16.0MB/s]                           


# Exctracting f0s

In [4]:
#Collecting audio file names 
all_files = []
for root, dirs, files in os.walk('/root/mir_datasets/orchset/audio/mono/'):
    for file in files:
        if file.endswith('.wav'):
            file_name = os.path.join(root,file)
            all_files.append(file_name)

In [6]:
for file in all_files:
 x, fs = sf.read(file)
 x = x / np.max(np.abs(x))#amplitude normalisation
 minF0 = 50 #in Hz
 maxF0 = 2000 #in Hz
 windowSize = 4096
 hopSize = 1024
 w = get_window('blackman', windowSize)
 startIndexes = np.arange(0, x.size - windowSize, hopSize, dtype = int)
 numWindows = startIndexes.size
 Audio(x, rate=fs)
 run_predominant_melody = PitchMelodia(guessUnvoiced = True,
                                      frameSize = windowSize,
                                      hopSize = hopSize,
                                      maxFrequency = maxF0,
                                      minFrequency = minF0,
                                      sampleRate = fs);
 f0, confidence = run_predominant_melody(x)

# Using essentia-melody algorithm

In [7]:
#Collecting mel file names for Annotations 
all_midi = []
for root, dirs, files in os.walk('/root/mir_datasets/orchset/GT'):
    for file in files:
        if file.endswith('.mel'):
            file_name = os.path.join(root,file)
            all_midi.append(file_name)        
            #Reading ground-truth
            ref_time, ref_freq = mir_eval.io.load_time_series(file)

#Saving estimated values to file
for file in all_files:
 fileNameF0 = file.replace('.wav','.f0_preDomMel.txt')
 startIndexes = np.arange(0, x.size - windowSize, hopSize, dtype = int)
 with open(fileNameF0, 'w') as f:
    for k in range(startIndexes.size):
        time = (startIndexes[k] + windowSize / 2) / fs
        f.write(str(time) + '\t' + str(f0[k]) + '\n')

#Collecting text file names 
all_text = []
for root, dirs, files in os.walk('/root/mir_datasets/orchset/audio/mono/'):
    for file in files:
        if file.endswith('.txt'):
            file_name = os.path.join(root,file)
            all_text.append(file_name)

for file in all_text:
  #Re-reading Essentia estimation results
  est_time, est_freq = mir_eval.io.load_time_series(file) 

In [ ]:
for file in files
  #Running evaluation and ptinting
  scores = mir_eval.melody.evaluate(ref_time, ref_freq,est_time, est_freq)
  for score in scores:
    print(score,'\t',scores[score])

Voicing Recall 	 0.7893447642375995
Voicing False Alarm 	 0.7719298245614035
Raw Pitch Accuracy 	 0.004286589099816289
Raw Chroma Accuracy 	 0.07164727495407226
Overall Accuracy 	 0.011834319526627219


# Using crepe algorithm

In [8]:
!pip install crepe
import crepe

     |████████████████████████████████| 368kB 6.4MB/s 
  Created wheel for crepe: filename=crepe-0.0.11-cp36-none-any.whl size=134848478 sha256=80b4ec0c41c2cbf6312d08058b6bb277fb082da54415d477bf03e1cda7babfd0
  Stored in directory: /root/.cache/pip/wheels/7f/68/d0/502d27bfe590bfa51c6fe95cf9e4482fed18c22c480a8d7c77
Successfully built crepe


In [ ]:
# After deleting the Readme.txt in orchest folder we can run the following
scores =[]
for root, dirs, files in os.walk('/root/mir_datasets/orchset/'):
        if file.endswith('.mel'):
            file_name = os.path.join(root,file)
            all_midi.append(file_name)        
            #Reading ground-truth
            ref_time, ref_freq = mir_eval.io.load_time_series(file)
        elif file.endswith('.wav'):
           hopSize = 441#10 miliseconds
           windowSize = 4096
           fs = 44100
           x, fs = sf.read(file)
           x = x / np.max(np.abs(x))#amplitude normalisation
           #x = EqualLoudness()(x)
           est_time, est_freq, confidence, activation = crepe.predict(x, fs,viterbi=True)
           #Running evaluation and ptinting
           for score in scores:
             scores[score] = mir_eval.melody.evaluate(ref_time, ref_freq,est_time, est_freq)
             print(score,'\t',scores[file])

27/69 [==========>...................] - ETA: 1:08